### Practice Activity 7.1: Cross-Validation & Tuning

Elizabeth Berry

11/10/23

In [154]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.compose import ColumnTransformer

In [155]:
lr = LinearRegression()

ames = pd.read_csv("/Users/elleberry/Desktop/Classes/MBA/GSB 544 - Computing and Machine Learning/Data/AmesHousing.csv")

X = ames[['Gr Liv Area', 'TotRms AbvGrd', 'Bldg Type']]
y = ames['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [156]:
# Model 1: Size and number of rooms.
model_1 = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

In [157]:
# Model 2: Size, number of rooms, and building type.
model_2 = Pipeline([
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), ['Gr Liv Area', 'TotRms AbvGrd']),
            ('cat', OneHotEncoder(sparse_output=False), ['Bldg Type'])
        ],
        remainder='passthrough'
    )),
    ('regressor', LinearRegression())
])

In [158]:
# Model 3: Size, building type, and their interaction.
model_3 = Pipeline([
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), ['Gr Liv Area']),
            ('cat', OneHotEncoder(sparse_output=False), ['Bldg Type'])  # Use OneHotEncoder to handle the categorical variable
        ],
        remainder='passthrough'
    )),
    ('poly', PolynomialFeatures(interaction_only=True)),
    ('regressor', LinearRegression())
])

In [159]:
# Model 4: 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type.
model_4 = Pipeline([
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), ['Gr Liv Area', 'TotRms AbvGrd']),
            ('cat', OneHotEncoder(sparse_output=False), ['Bldg Type'])
        ],
        remainder='passthrough'
    )),
    ('poly', PolynomialFeatures(degree=5)),
    ('regressor', LinearRegression())
])

In [160]:
model_1.fit(X[['Gr Liv Area', 'TotRms AbvGrd']], y)
model_2.fit(X, y)
model_3.fit(X, y)
model_4.fit(X, y)

pred_1 = model_1.predict(X_test[['Gr Liv Area', 'TotRms AbvGrd']])
pred_2 = model_2.predict(X_test)
pred_3 = model_3.predict(X_test)
pred_4 = model_4.predict(X_test)

rmse_1 = np.sqrt(mean_squared_error(y_test, pred_1))
rmse_2 = np.sqrt(mean_squared_error(y_test, pred_2))
rmse_3 = np.sqrt(mean_squared_error(y_test, pred_3))
rmse_4 = np.sqrt(mean_squared_error(y_test, pred_4))

print("RMSE for Model 1: ", rmse_1)
print("RMSE for Model 2: ", rmse_2)
print("RMSE for Model 3: ", rmse_3)
print("RMSE for Model 4: ", rmse_4)

RMSE for Model 1:  55737.57021055191
RMSE for Model 2:  53918.79395409218
RMSE for Model 3:  52784.05253835477
RMSE for Model 4:  49490.500408507454


Model 4 is the best for the data as it results in the lowest Root Mean Square Error of all 4 models. 

In [161]:
# Cross-Validation
from sklearn.model_selection import cross_val_score

X = ames[['Gr Liv Area', 'TotRms AbvGrd', 'Bldg Type']].copy()
X['Bldg Type'] = X['Bldg Type'].astype(str)

X_test['Bldg Type'] = X_test['Bldg Type'].astype(str)

# Dummify Building Type in the test set
X_test = pd.get_dummies(X_test, columns=['Bldg Type'], drop_first=True) 

# Scoring method
neg_mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Cross-validated RMSE
rmse_1 = np.sqrt(-cross_val_score(model_1, X[['Gr Liv Area', 'TotRms AbvGrd']], y, cv=5, scoring=neg_mse_scorer).mean())
rmse_2 = np.sqrt(-cross_val_score(model_2, X, y, cv=5, scoring=neg_mse_scorer).mean())
rmse_3 = np.sqrt(-cross_val_score(model_3, X, y, cv=5, scoring=neg_mse_scorer).mean())
rmse_4 = np.sqrt(-cross_val_score(model_4, X, y, cv=5, scoring=neg_mse_scorer).mean())

print("Cross-validated RMSE for Model 1: ", rmse_1)
print("Cross-validated RMSE for Model 2: ", rmse_2)
print("Cross-validated RMSE for Model 3: ", rmse_3)
print("Cross-validated RMSE for Model 4: ", rmse_4)

Cross-validated RMSE for Model 1:  56001.24023779208
Cross-validated RMSE for Model 2:  54305.840806452674
Cross-validated RMSE for Model 3:  53529.853688347794
Cross-validated RMSE for Model 4:  589320.4825004


After finding the cross-validated root mean squared error it appears that model 4 is now very unfit for the data and model 3 is the best option. 

In [162]:
# Tuning
    # Consider one hundred modeling options for house price:
        # House size, trying degrees 1 through 10
        # Number of rooms, trying degrees 1 through 10
        # Building Type
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

# Dummify Building Type
X = pd.get_dummies(X, columns=['Bldg Type'], drop_first=True)

best_degree_combination = None
best_rmse = float('inf')

for degree_size in range(1, 11):
    for degree_rooms in range(1, 11):
        # Define the degrees for polynomial features
        degrees = {
            'preprocessing__polynomial__degree': [degree_size, degree_rooms]
        }

        ct_poly = ColumnTransformer(
            [
                ("polynomial", PolynomialFeatures(), ['Gr Liv Area', 'TotRms AbvGrd'])
            ],
            remainder='passthrough'
        )

        lr_pipeline_poly = Pipeline([
            ("preprocessing", ct_poly),
            ("linear_regression", LinearRegression())
        ]).set_output(transform="pandas")

        # GridSearchCV to find best model
        gscv = GridSearchCV(lr_pipeline_poly, degrees, cv=5, scoring='neg_mean_squared_error')

        gscv_fitted = gscv.fit(X, y)

        neg_mse = -gscv_fitted.best_score_

        if neg_mse < best_rmse:
            best_rmse = neg_mse
            best_degree_combination = (degree_size, degree_rooms)

# After finding the best degree combination, create a model with it
best_degree_size, best_degree_rooms = best_degree_combination
best_ct_poly = ColumnTransformer(
    [
        ("poly_size", PolynomialFeatures(degree=best_degree_size), ['Gr Liv Area']),
        ("poly_rooms", PolynomialFeatures(degree=best_degree_rooms), ['TotRms AbvGrd']),
    ],
    remainder='passthrough'
)

best_lr_pipeline_poly = Pipeline([
    ("preprocessing", best_ct_poly),
    ("linear_regression", LinearRegression())
]).set_output(transform="pandas")

best_lr_pipeline_poly.fit(X, y)

# Make predictions with the best model
y_pred = best_lr_pipeline_poly.predict(X)

# Calculate R^2 for the best combination
best_r2 = r2_score(y, y_pred)

print("Best Degree Combination (Gr Liv Area, TotRms AbvGrd):", best_degree_combination)
print("Best Negative Mean Squared Error:", best_rmse)
print("Best R^2 Score:", best_r2)

Best Degree Combination (Gr Liv Area, TotRms AbvGrd): (1, 3)
Best Negative Mean Squared Error: 2909000559.982959
Best R^2 Score: 0.5547887744591438


Q1: House size with degree 1, number of rooms with degree 10, building Type 
Q2: It can be time consuming to test all the models especially with large datasets, and trying many different combinations can lead to overfitting. 